In [1]:
import nltk
from nltk.corpus import webtext
import numpy as np
from __future__ import division
import vincent
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
import pandas as pd

In [2]:
poc = webtext.raw("pirates.txt")
p = poc.split('\n')[2:-2]
#for s in p:
#    print s

In [3]:
# The raw text is inconsistent. In this section I'll fix it.

# Add the missing 'Scene:' on the line 297, delete the line there since it's not important.
p[297] = "Scene: Port Royal"

# There is a "Prisoners" on the line 300 that has to be changed to "PRISONERS".
aux = list(p[300])
aux[0:9] = "PRISONERS"
p[300] = "".join(aux)

# Add the missing 'Scene:' on the line 315.
p[315] = "Scene: " + p[315]

# Add the missing 'Scene:' on the line 338 and 339.
p[338] = p[338][0:-9]
p[339] = "Scene: Longboat"

# Change "###PINTEL" for "PINTEL".
p[358] = p[358][3:-1]

# Change "LEEECH" for "LEECH"
p[421] = "LEECH: Stop! ..."

# Add the missing 'Scene:' on the line 425.
p[425] = "Scene: " + p[425]

# Add the missing 'Scene:' on the line 552.
p[552] = "Scene: PANTANO RIVER"

# Add the missing 'Scene:' on the line 563.
p[563] = "Scene: " + p[563]

# Add the missing 'Scene:' on the line 665.
p[665] = "Scene: " + p[563]

# Add the missing 'Scene:' on the line 691.
p[691] = "Scene: " + p[691]

# Add the missing 'Scene:' on the line 717.
p[717] = "Scene: BLACK PEARL"

# Add the missing 'Scene:' on the line 777.
p[777] = "Scene: TORTUGA"

# Add the missing 'Scene:' on the line 833.
p[833] = "Scene: FLYING DUTCHMAN"

# Add the missing 'Scene:' on the line 863.
p[863] = "Scene: " + p[863]

# Add the missing 'Scene:' on the line 935.
p[935] = "Scene: " + p[935]

# Add the missing 'Scene:' on the line 945.
p[945] = "Scene: " + p[945]

# TODO: the scene Edinburgh Trader on line 1028, difficult to place.

# Add the missing 'Scene:' on the line 1046.
p[1046] = "Scene: " + p[1046]

# Add the missing 'Scene:' on the line 1088.
p[1088] = "Scene: BLACK PEARL"

# Add the missing 'Scene:' on the line 1129.
p[1129] = "Scene: " + p[1129]

# Add the missing: 'Scene:' on the line 1143.
p[1143] = "Scene: " + p[1143]

# Add the missing: 'Scene:' on the line 1154.
p[1154] = "Scene: " + p[1154]

# Add the missing 'Scene:' on the line 1224.
p[1224] = "Scene: " + p[1224]

# Add the missing 'Scene:' on the line 1290.
p[1290] = "Scene: " + p[1290]

# Add the missing 'Scene:' on the line 1327.
p[1327] = "Scene: BLACK PEARL"

# Add the missing 'Scene:' on the line 1363.
p[1363] = "Scene: " + p[1363]

# Add the missing 'Scene:' on the line 1482.
p[1482] = "Scene:" + p[1482]

# Add the missing 'Scene:' on the line 1490.
p[1490] = "Scene: " + p[1490]

# Add the missing 'Scene:' on the line 1500.
p[1500] = "Scene: " + p[1500]

# Add a 'Scene' to the end to make the processing easier.
p[-1] = "Scene: THE END"

In [4]:
# Remove the lines that start with '['
p4 = [s for s in p if s.split(':')[0] != "["]
#p4

In [5]:
# Get the characters of the story.

characters = set()
for s in p4:
    if ":" in s:
        c = s.split(':')[0]
        characters.add(c)
    
characters = list(characters)

# Remove the outliers.
characters.remove('Scene')
characters.remove('[Among the crew')
characters.remove('FLYING DUTCHMAN')

print len(characters)
print

for elem in characters:
    print elem

48

MARTY
GIBBS
CANNIBAL CROWD
CANNIBAL BOY
CANNIBAL WOMAN
HADRUS
JACK SPARROW
SHRIMPER'S BROTHER
COTTON'S PARROT
ELIZABETH SWANN
WYVERN
KOLENIKO
HELMSMAN
CHAPLAIN
SHRIMPER
WILL TURNER
WOUNDED SAILOR
TIA DALMA
RAGETTI
GISELLE
ONE-ARMED MAN
TURKISH FISHERMAN
DAVY JONES
GUARD
CARRUTHERS
ROMANTIC MAN
ELDERLY MAN
?
QUARTERMASTER
LORD CUTLER BECKETT
BARBOSSA
MERCER
LEECH
GREENBEARD
BELLAMY
MESSENGER SAILOR
WEATHERED SAILOR
PINTEL
BO'SUN
PRISONERS
HALF-BLIND FISHERMAN
CREW
BURSAR
JILTED MAN
GREEK FISHERMAN
MACCUS
NORRINGTON
SCARLETT


In [10]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

# Add the nodes
for char in characters:
    G.add_node(char)

In [11]:
# Add the edges.

inte = []
for sen in p4:
    if sen[:6] == "Scene:":
        if len(inte) != 0:
            for i, c in enumerate(inte):    
                for j in range(i+1,len(inte)):
                    if G.has_edge(c, inte[j]):
                        G[c][inte[j]]['weight'] = G[c][inte[j]]['weight'] + 1
                    else:
                        G.add_edge(c, inte[j], weight=1)

            inte = []
    elif ":" in sen:
        c = sen.split(":")[0]
        if c in characters and c not in inte:
            inte.append(c)

                    
# Remove the nodes with degree 0.
for char in characters:
    if G.degree(char) == 0:
        G.remove_node(char)
        print char
        
        
# Add the degree to each node.
for char in characters:
    if G.has_node(char):
        G.node[char]['deg'] = G.degree(char)


In [8]:
#draw the graph
pos = nx.circular_layout(G)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_labels(G, pos)
nx.draw_networkx_edges(G, pos)

#show
plt.savefig("POC5.png")
plt.close()

In [12]:
x = nx.algorithms.pagerank(G)

import operator
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[(u'WILL TURNER', 0.10466822912255654),
 (u'JACK SPARROW', 0.08507554614130902),
 (u'ELIZABETH SWANN', 0.07367765636927222),
 (u'GIBBS', 0.0682169301575914),
 (u'PINTEL', 0.05535900409519341),
 (u'RAGETTI', 0.050632439060312645),
 (u'NORRINGTON', 0.04067579179294711),
 (u'DAVY JONES', 0.03084870984476889),
 (u"COTTON'S PARROT", 0.026999193178829598),
 (u'GREEK FISHERMAN', 0.020833333333333332)]

In [10]:
import community

com = community.best_partition(G)

sorted_x = sorted(com.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x

[(u'BURSAR', 7),
 (u'MESSENGER SAILOR', 7),
 (u'BELLAMY', 7),
 (u'QUARTERMASTER', 7),
 (u'PRISONERS', 6),
 (u'MERCER', 6),
 (u'LORD CUTLER BECKETT', 6),
 (u'GUARD', 6),
 (u'GREEK FISHERMAN', 5),
 (u'TURKISH FISHERMAN', 5),
 (u'JILTED MAN', 4),
 (u'ELDERLY MAN', 4),
 (u'ROMANTIC MAN', 4),
 (u'ONE-ARMED MAN', 4),
 (u'MACCUS', 3),
 (u"BO'SUN", 3),
 (u'GREENBEARD', 3),
 (u'CHAPLAIN', 3),
 (u'DAVY JONES', 3),
 (u'WOUNDED SAILOR', 3),
 (u'KOLENIKO', 3),
 (u'HELMSMAN', 3),
 (u'CANNIBAL BOY', 2),
 (u'CANNIBAL WOMAN', 2),
 (u'SCARLETT', 1),
 (u'WEATHERED SAILOR', 1),
 (u"SHRIMPER'S BROTHER", 1),
 (u'GISELLE', 1),
 (u'SHRIMPER', 1),
 (u'HALF-BLIND FISHERMAN', 1),
 (u'NORRINGTON', 0),
 (u'CREW', 0),
 (u'HADRUS', 0),
 (u'PINTEL', 0),
 (u'LEECH', 0),
 (u'BARBOSSA', 0),
 (u'CARRUTHERS', 0),
 (u'?', 0),
 (u'RAGETTI', 0),
 (u'WILL TURNER', 0),
 (u'WYVERN', 0),
 (u'ELIZABETH SWANN', 0),
 (u"COTTON'S PARROT", 0),
 (u'TIA DALMA', 0),
 (u'JACK SPARROW', 0),
 (u'CANNIBAL CROWD', 0),
 (u'GIBBS', 0),
 (u'MAR

In [11]:
nx.write_gexf(G, "POC5.gexf")

In [17]:
x = nx.algorithms.eigenvector_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[(u'JACK SPARROW', 0.4271448335262501),
 (u'WILL TURNER', 0.4057909261851892),
 (u'GIBBS', 0.37636874701879697),
 (u'ELIZABETH SWANN', 0.37290808669843756),
 (u'PINTEL', 0.36832848997207174),
 (u'RAGETTI', 0.3439285311251907),
 (u'NORRINGTON', 0.21461284927533486),
 (u"COTTON'S PARROT", 0.11130771708326352),
 (u'MARTY', 0.10886226100825261),
 (u'DAVY JONES', 0.09218498081593503)]

In [18]:
x = nx.algorithms.degree_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
#for elem in sorted_x[:10]:
#    print "(" + str(elem[0]) + ", " + str(elem[1]*(nx.number_of_nodes(G)-1)) + "),"
sorted_x[:10]

[(u'WILL TURNER', 0.6808510638297872),
 (u'JACK SPARROW', 0.5319148936170213),
 (u'ELIZABETH SWANN', 0.5106382978723404),
 (u'GIBBS', 0.44680851063829785),
 (u'NORRINGTON', 0.36170212765957444),
 (u"COTTON'S PARROT", 0.2765957446808511),
 (u'PINTEL', 0.2553191489361702),
 (u'DAVY JONES', 0.2553191489361702),
 (u'RAGETTI', 0.2553191489361702),
 (u'MARTY', 0.2127659574468085)]

In [20]:
x = nx.algorithms.closeness_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[(u'WILL TURNER', 0.7428466617754953),
 (u'JACK SPARROW', 0.6244218316373729),
 (u'ELIZABETH SWANN', 0.615501519756839),
 (u'GIBBS', 0.5902069367531333),
 (u'NORRINGTON', 0.5595468361425808),
 (u"COTTON'S PARROT", 0.5595468361425808),
 (u'PINTEL', 0.5254281266216918),
 (u'DAVY JONES', 0.5254281266216918),
 (u'RAGETTI', 0.5254281266216918),
 (u'MARTY', 0.5129179331306991)]

In [15]:
x = nx.algorithms.betweenness_centrality(G)

sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x.reverse()
sorted_x[:10]

[(u'WILL TURNER', 0.4392961596939395),
 (u'JACK SPARROW', 0.15531521105064122),
 (u'SHRIMPER', 0.1517113783533765),
 (u'ELIZABETH SWANN', 0.12041259696208817),
 (u"COTTON'S PARROT", 0.05530012694675045),
 (u'GIBBS', 0.05128014651603921),
 (u'DAVY JONES', 0.03823620104841195),
 (u'NORRINGTON', 0.03710048816431794),
 (u'LORD CUTLER BECKETT', 0.007873368280398809),
 (u'MARTY', 0.0042451073440897685)]

In [6]:
import collections

# Characters with most lines.
chl = []
for sen in p4:
    if ":" in sen:
        c = sen.split(":")[0]
        if c in characters:
            chl.append(c)

counter = collections.Counter(chl)
for elem in counter.most_common(10):
    print elem


(u'JACK SPARROW', 193)
(u'WILL TURNER', 133)
(u'ELIZABETH SWANN', 93)
(u'GIBBS', 85)
(u'DAVY JONES', 46)
(u'PINTEL', 39)
(u'LORD CUTLER BECKETT', 34)
(u'RAGETTI', 28)
(u'NORRINGTON', 28)
(u'TIA DALMA', 26)


In [7]:
pre_data = counter.most_common(10)

vincent.core.initialize_notebook()
data = [x[1] for x in pre_data]
index = [x[0] for x in pre_data]
s = pd.Series(data=data, index=index)


bars = vincent.Bar(s)
bars.colors(brew='Pastel2')
bars.axis_titles(x='Character', y='# lines')
bars.scales['x'].padding = 0.1
bars.display()

In [8]:
# Characters in most amount of scenes.

chs = []
inte = []
for sen in p4:
    if sen[:6] == "Scene:":
        if len(inte) != 0:
            chs.extend(inte)
            inte = []
    elif ":" in sen:
        c = sen.split(":")[0]
        if c in characters and c not in inte:
            inte.append(c)
            
counter = collections.Counter(chs)
for elem in counter.most_common(10):
    print elem


(u'WILL TURNER', 26)
(u'JACK SPARROW', 24)
(u'GIBBS', 19)
(u'ELIZABETH SWANN', 16)
(u'PINTEL', 14)
(u'RAGETTI', 12)
(u'DAVY JONES', 10)
(u'NORRINGTON', 7)
(u'LORD CUTLER BECKETT', 5)
(u'MACCUS', 5)


In [9]:
pre_data = counter.most_common(10)

vincent.core.initialize_notebook()
data = [x[1] for x in pre_data]
index = [x[0] for x in pre_data]
s = pd.Series(data=data, index=index)


bars = vincent.Bar(s)
bars.colors(brew='Pastel2')
bars.axis_titles(x='Character', y="# scenes")
bars.scales['x'].padding = 0.1
bars.display()

In [29]:
# Most used words of the main characters.

# Most used words of Jack Sparrow.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "JACK SPARROW":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'want', 12),
 (u'come', 11),
 (u'know', 9),
 (u'oh', 9),
 (u'love', 8),
 (u'dirt', 8),
 (u'bugger', 8),
 (u"you're", 7),
 (u'hey', 7),
 (u'captain', 6)]

In [31]:
# Most used words of Will Turner.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "WILL TURNER":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
#freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'jack', 22),
 (u'find', 8),
 (u'get', 7),
 (u'elizabeth', 6),
 (u'euh', 5),
 (u'know', 5),
 (u'hold', 5),
 (u'key', 5),
 (u'come', 5),
 (u"i'm", 5)]

In [32]:
# Most used words of Elizabeth Swann.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "ELIZABETH SWANN":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'jack', 11),
 (u'find', 7),
 (u'know', 7),
 (u'oh', 7),
 (u"you're", 6),
 (u'want', 6),
 (u"i'm", 5),
 (u'something', 5),
 (u'man', 5),
 (u'sparrow', 4)]

In [33]:
# Most used words of Gibbs.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "GIBBS":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'aye', 9),
 (u'jack', 9),
 (u'us', 7),
 (u'sea', 5),
 (u"there's", 5),
 (u'ho', 5),
 (u'bit', 4),
 (u'like', 4),
 (u'got', 4),
 (u'think', 4)]

In [34]:
# Most used words of Davy Jones.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "DAVY JONES":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'ha', 12),
 (u'sparrow', 4),
 (u'captain', 4),
 (u'jack', 4),
 (u'let', 4),
 (u'huh', 4),
 (u'liar', 3),
 (u'years', 3),
 (u'fear', 3),
 (u'chest', 3)]

In [36]:
# Most used words of Lord Cutler Beckett.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "LORD CUTLER BECKETT":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
#freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'jack', 7),
 (u'compass', 5),
 (u'sparrow', 5),
 (u'mister', 4),
 (u'one', 4),
 (u'world', 3),
 (u"there's", 3),
 (u'freedom', 3),
 (u"i'm", 3),
 (u'currency', 3)]

In [38]:
# Most used words of Pintel.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "PINTEL":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
#freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'aye', 3),
 (u"it's", 3),
 (u"ain't", 3),
 (u'us', 3),
 (u'lines', 3),
 (u'mooring', 3),
 (u"don't", 2),
 (u'back', 2),
 (u"o'", 2),
 (u'enough', 2)]

In [39]:
# Most used words of Ragetti.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "RAGETTI":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u"don't", 3),
 (u"it's", 3),
 (u'unh-unh', 3),
 (u'well', 3),
 (u'souls', 2),
 (u"he's", 2),
 (u'save', 2),
 (u'ha-ha-ha', 2),
 (u'thief/thing', 2),
 (u'back', 2)]

In [41]:
# Most used words of Norrington.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "NORRINGTON":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
#freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'oh', 4),
 (u'one', 3),
 (u'life', 3),
 (u'kill', 2),
 (u'captain', 2),
 (u'mister', 2),
 (u"don't", 2),
 (u'actually', 2),
 (u'sorry', 2),
 (u'come', 2)]

In [43]:
# Most used words of Tia Dalma.
words = []
for sen in p4:
    if ":" in sen and sen[:6] != "Scene:":
        c = sen.split(":")[0]
        if c == "TIA DALMA":
            aux = sen.split(":")[1]
            for x in aux.split():
                words.append(x.lower().strip("!?,.:;-_*"))
                
filtered_words = [w for w in words if not w in stopwords.words("english")]

freq_dist = nltk.FreqDist(filtered_words)
#freq_dist.pop('')
fw = freq_dist.most_common(10)
fw

[(u'de', 8),
 (u'know', 7),
 (u'hmmm', 4),
 (u'jack', 4),
 (u'would', 4),
 (u'wid', 4),
 (u'chest', 4),
 (u'sparrow', 3),
 (u'cannot', 3),
 (u'back', 3)]